# iLykei Lecture Series

# Advanced Machine Learning and Artificial Intelligence

# Reinforcement Learning

## Notebook 6: Learning Ms. Pac-Man with DQN

## Yuri Balasanov, Mihail Tselishchev, &copy; iLykei 2018

##### Main text: Hands-On Machine Learning with Scikit-Learn and TensorFlow, Aurelien Geron, &copy; Aurelien Geron 2017, O'Reilly Media, Inc


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt


import numpy as np
import random
import time
import os
import gc

from keras.models import Sequential, clone_model
from keras.layers import Dense, Flatten, Conv2D, InputLayer
from keras.callbacks import CSVLogger, TensorBoard
from keras.optimizers import Adam
import keras.backend as K

import gym

plt.rcParams['figure.figsize'] = (9, 9)

Using TensorFlow backend.


# Deep Q-Learning of MS. Pac-Man with Keras

This notebook shows how to implement a deep neural network approach to train an agent to play Ms.Pac-Man Atari game.


## Explore the game

Use [Gym](https://gym.openai.com/) toolkit that provides both game environment and also a convenient renderer of the game.

Create an environment.

In [3]:
env = gym.make("MsPacman-ram-v0")
env.action_space  # actions are integers from 0 to 8

Discrete(9)

Try to play the game using random strategy:

In [3]:
env.reset()
done = False
score = 0
while not done:
    action = random.randrange(env.action_space.n)  # select random action
    obs, reward, done, info = env.step(action)     # make action and get results
    score += reward
    env.render()
    time.sleep(0.01)
    
env.close()
print('Score =', score)

Score = 160.0


### Observation

In this environment, observation (i.e. current state) is the RAM of the Atari machine, namely a vector of 128 bytes:

In [4]:
obs = env.reset()
print('obs shape =', obs.shape)
print('obs dtype =', obs.dtype)

obs shape = (128,)
obs dtype = uint8


Look at that vector:

In [5]:
print(obs)

[  0 112 114 115   0   3  88  88  88  88  88   0  80  80  80  50  98   0
   0   3   0   0   1   0   0   1   6   6 198   4  71   0  45   1   0 198
 198   0   0   0   0  16  52   0   0 120   0 100 130   0   0 134   1 222
   0   1   3   0   6  80 255 255   0 255 255  80 255 255  80 255 255  80
 255 255  80 191 191  80 191 191  80 191 191  80 255 255  80 255 255  80
 255 255  80 255 255   0 255 255  80 255 255  20 223  43 217 123 217 123
 217 123 217 123 217 123 217 221   0  63   0   0   0   0   0   2  66 240
 146 215]


Create a deep neural network that takes byte vector as an input and produces Q-values for state-action pairs.

## Creating a DQN-model using Keras

The following model is of the same general type applied to the cartPole problem.

Use vanilla multi-layer dense network with relu activations which computes Q-values $Q(s,a)$ for all states $s$ and actions $a$ (with some discount factor $\gamma$).
This neural network denoted by $Q(s\ |\ \theta)$ takes current state as an input and produces a vector of q-values for all 9 possible actions. Vector $\theta$ corresponds to all trainable parameters.

In [7]:
# def create_dqn_model(input_shape, nb_actions, dense_layers, dense_units):
#     model = Sequential()
#     model.add(InputLayer(input_shape=input_shape))
#     for i in range(dense_layers):
#         model.add(Dense(units=dense_units, activation='relu'))
#     model.add(Dense(nb_actions, activation='linear'))
#     return model

def create_dqn_model(input_shape, nb_actions, dense_layers, dense_units):
    model = Sequential()
    for i in range(dense_layers):
        if i==0:
            model.add(Dense(units=dense_units, activation='relu',input_shape=input_shape))
        else:
            model.add(Dense(units=dense_units, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model

Create a network using specific input shape and action space size. We call this network *online*.

In [8]:
input_shape = obs.shape
nb_actions = env.action_space.n  # 9
dense_layers = 5
dense_units = 256

online_network = create_dqn_model(input_shape, nb_actions, dense_layers, dense_units)
online_network.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_6 (Dense)              (None, 9)                 2313      
Total params: 298,505
Trainable params: 298,505
Non-trainable params: 0
_________________________________________________________________


In [8]:
from keras.utils import plot_model
plot_model(online_network, to_file='online_DenseNetwork.png',show_shapes=True,show_layer_names=True)

Plot the architecture of the network saved as *online_DenseNetwork.png*. (To see the plot log into iLykei.com, then rerun this cell).

![Model plot](https://ilykei.com/api/fileProxy/documents%2FAdvanced%20Machine%20Learning%2FReinforced%20Learning%2Fonline_DenseNetwork.png)

This network is used to explore states and rewards of Markov decision process according to an $\varepsilon$-greedy exploration strategy:

In [9]:
def epsilon_greedy(q_values, epsilon, n_outputs):
    if random.random() < epsilon:
        return random.randrange(n_outputs)  # random action
    else:
        return np.argmax(q_values)          # q-optimal action

Online network stores explored information in a *replay memory*, a double-ended queue (deque).

In [10]:
from collections import deque

replay_memory_maxlen = 1000000
replay_memory = deque([], maxlen=replay_memory_maxlen)

So, online network explores the game using $\varepsilon$-greedy strategy and saves experienced transitions in replay memory. 

In order to produce Q-values for $\varepsilon$-greedy strategy, following the proposal of the [original paper by Google DeepMind](https://www.nature.com/articles/nature14236), use another network, called *target network*, to calculate "ground-truth" target for the online network. *Target network*, has the same architecture as online network and is not going to be trained. Instead, weights from the online network are periodically copied to target network.

In [11]:
target_network = clone_model(online_network)
target_network.set_weights(online_network.get_weights())

The target network uses past experience in the form of randomly selected records of the replay memory to predict targets for the online network: 

- Select a random minibatch from replay memory containing tuples $(\text{state},\text{action},\text{reward},\text{next_state})$

- For every tuple $(\text{state},\text{action},\text{reward},\text{next_state})$ from minibatch Q-value function $Q(\text{state},\text{action}\ |\ \theta_{\text{online}})$ is trained on predictions of $Q(\text{next_state}, a\ |\ \theta_\text{target})$ according to Bellman-type equation: 

$$y_\text{target} = \text{reward} + \gamma \cdot \max_a Q(\text{next_state}, a\ |\ \theta_\text{target})$$
if the game continues and $$ y_\text{target} = \text{reward}$$ if the game has ended. 

Note that at this step predictions are made by the target network. This helps preventing situations when online network simultaneously predicts values and creates targets, which might potentially lead to instability of training process.

- For each record in the minibatch targets need to be calculated for only one specific $\text{action}$ output of online network. It is important to ignore all other outputs during optimization (calculating gradients). So, predictions for every record in the minibatch are calculated by online network first, then the values corresponding to the actually selected action are replaced with ones predicted by target network. 

## Double DQN

Approach proposed in the previous section is called **DQN**-approach. 

DQN approach is very powerful and allows to train agents in very complex, very multidimentional environments.

However, [it is known](https://arxiv.org/abs/1509.06461) to overestimate q-values under certain conditions. 

Alternative approach proposed in the [same paper](https://arxiv.org/abs/1509.06461) is called **Double DQN**. 

Instead of taking action that maximizes q-value for target network, they pick an action that maximizes q-value for online network as an optimal one:

$$y_\text{target} = \text{reward} + \gamma \cdot Q\left(\text{next_state}, \arg\max_a Q\left(\text{next_state},a\ |\ \theta_\text{online}\right)\ |\ \theta_\text{target}\right).$$


## Training DQN

First, define hyperparameters (Do not forget to change them before moving to cluster):

In [12]:
name = 'MsPacman_DQN'  # used in naming files (weights, logs, etc)
n_steps = 10000        # total number of training steps (= n_epochs)
warmup = 1000          # start training after warmup iterations
training_interval = 4  # period (in actions) between training steps
save_steps = int(n_steps/10)  # period (in training steps) between storing weights to file
copy_steps = 100       # period (in training steps) between updating target_network weights
gamma = 0.9            # discount rate
skip_start = 90        # skip the start of every game (it's just freezing time before game starts)
batch_size = 64        # size of minibatch that is taken randomly from replay memory every training step
double_dqn = False     # whether to use Double-DQN approach or simple DQN (see above)
# eps-greedy parameters: we slowly decrease epsilon from eps_max to eps_min in eps_decay_steps
eps_max = 1.0
eps_min = 0.05
eps_decay_steps = int(n_steps/2)

learning_rate = 0.001

Compile online-network with Adam optimizer, mean squared error loss and `mean_q` metric, which measures the maximum of predicted q-values averaged over samples from minibatch (we expect it to increase during training process).

In [13]:
def mean_q(y_true, y_pred):
    return K.mean(K.max(y_pred, axis=-1))


online_network.compile(optimizer=Adam(learning_rate), loss='mse', metrics=[mean_q])

Create folder for logs and trained weights:

In [14]:
if not os.path.exists(name):
    os.makedirs(name)
    
weights_folder = os.path.join(name, 'weights')
if not os.path.exists(weights_folder):
    os.makedirs(weights_folder)

Use standard callbacks:

In [15]:
csv_logger = CSVLogger(os.path.join(name, 'log.csv'), append=True, separator=';')
tensorboard = TensorBoard(log_dir=os.path.join(name, 'tensorboard'), write_graph=False, write_images=False)

Next chunk of code explores the game, trains online network and periodically copies weights to target network as explained above.

In [16]:
# counters:
step = 0          # training step counter (= epoch counter)
iteration = 0     # frames counter
episodes = 0      # game episodes counter
done = True       # indicator that env needs to be reset

episode_scores = []  # collect total scores in this list and log it later

while step < n_steps:
    if done:  # game over, restart it
        obs = env.reset()
        score = 0  # reset score for current episode
        for skip in range(skip_start):  # skip the start of each game (it's just freezing time before game starts)
            obs, reward, done, info = env.step(0)
            score += reward
        state = obs
        episodes += 1

    # Online network evaluates what to do
    iteration += 1
    q_values = online_network.predict(np.array([state]))[0]  # calculate q-values using online network
    # select epsilon (which linearly decreases over training steps):
    epsilon = max(eps_min, eps_max - (eps_max-eps_min) * step/eps_decay_steps)
    action = epsilon_greedy(q_values, epsilon, nb_actions)
    # Play:
    obs, reward, done, info = env.step(action)
    score += reward
    if done:
        episode_scores.append(score)
    next_state = obs
    # Let's memorize what just happened
    replay_memory.append((state, action, reward, next_state, done))
    state = next_state

    if iteration >= warmup and iteration % training_interval == 0:
        # learning branch
        step += 1
        minibatch = random.sample(replay_memory, batch_size)
        replay_state = np.array([x[0] for x in minibatch])
        replay_action = np.array([x[1] for x in minibatch])
        replay_rewards = np.array([x[2] for x in minibatch])
        replay_next_state = np.array([x[3] for x in minibatch])
        replay_done = np.array([x[4] for x in minibatch], dtype=int)

        # calculate targets (see above for details)
        if double_dqn == False:
            # DQN
            target_for_action = replay_rewards + (1-replay_done) * gamma * \
                                    np.amax(target_network.predict(replay_next_state), axis=1)
        else:
            # Double DQN
            best_actions = np.argmax(online_network.predict(replay_next_state), axis=1)
            target_for_action = replay_rewards + (1-replay_done) * gamma * \
                                    target_network.predict(replay_next_state)[np.arange(batch_size), best_actions]

        target = online_network.predict(replay_state)  # targets coincide with predictions ...
        target[np.arange(batch_size), replay_action] = target_for_action  #...except for targets with actions from replay
        
        # Train online network
        online_network.fit(replay_state, target, epochs=step, verbose=2, initial_epoch=step-1,
                           callbacks=[csv_logger, tensorboard])

        # Periodically copy online network weights to target network
        if step % copy_steps == 0:
            target_network.set_weights(online_network.get_weights())
        # And save weights
        if step % save_steps == 0:
            online_network.save_weights(os.path.join(weights_folder, 'weights_{}.h5f'.format(step)))
            gc.collect()  # also clean the garbage





Epoch 1/1
 - 4s - loss: 811.9525 - mean_q: 40.0516

Epoch 2/2
 - 0s - loss: 181.3541 - mean_q: 57.7502
Epoch 3/3
 - 0s - loss: 164.1998 - mean_q: 49.9461
Epoch 4/4
 - 0s - loss: 74.0074 - mean_q: 44.9145
Epoch 5/5
 - 0s - loss: 47.5185 - mean_q: 44.7768
Epoch 6/6
 - 0s - loss: 27.8570 - mean_q: 41.0398
Epoch 7/7
 - 0s - loss: 19.6247 - mean_q: 39.5150
Epoch 8/8
 - 0s - loss: 11.9747 - mean_q: 38.3906
Epoch 9/9
 - 0s - loss: 10.1756 - mean_q: 38.3837
Epoch 10/10
 - 0s - loss: 8.0550 - mean_q: 37.6850
Epoch 11/11
 - 0s - loss: 6.3151 - mean_q: 37.6307
Epoch 12/12
 - 0s - loss: 6.2599 - mean_q: 38.1350
Epoch 13/13
 - 0s - loss: 9.2713 - mean_q: 38.4844
Epoch 14/14
 - 0s - loss: 4.2630 - mean_q: 38.6195
Epoch 15/15
 - 0s - loss: 4.8821 - mean_q: 38.5510
Epoch 16/16
 - 0s - loss: 3.6426 - mean_q: 38.5114
Epoch 17/17
 - 0s - loss: 4.3575 - mean_q: 38.4802
Epoch 18/18
 - 0s - loss: 3.7726 - mean_q: 39.9176
Epoch 19/19
 - 0s - loss: 4.2404 - mean_q: 38.6411
Epoch 20/20
 - 0s - loss: 4.9785

 - 0s - loss: 0.9008 - mean_q: 37.0289
Epoch 142/142
 - 0s - loss: 0.6041 - mean_q: 35.8214
Epoch 143/143
 - 0s - loss: 2.4839 - mean_q: 36.3672
Epoch 144/144
 - 0s - loss: 0.4730 - mean_q: 35.6955
Epoch 145/145
 - 0s - loss: 1.2193 - mean_q: 36.4380
Epoch 146/146
 - 0s - loss: 1.1520 - mean_q: 35.8391
Epoch 147/147
 - 0s - loss: 0.8791 - mean_q: 36.3352
Epoch 148/148
 - 0s - loss: 0.6481 - mean_q: 35.8368
Epoch 149/149
 - 0s - loss: 0.9378 - mean_q: 35.9679
Epoch 150/150
 - 0s - loss: 0.7714 - mean_q: 35.9879
Epoch 151/151
 - 0s - loss: 1.0090 - mean_q: 36.1869
Epoch 152/152
 - 0s - loss: 0.9423 - mean_q: 35.9501
Epoch 153/153
 - 0s - loss: 1.0028 - mean_q: 36.2005
Epoch 154/154
 - 0s - loss: 0.8369 - mean_q: 35.7498
Epoch 155/155
 - 0s - loss: 1.6402 - mean_q: 36.0144
Epoch 156/156
 - 0s - loss: 0.8642 - mean_q: 36.0532
Epoch 157/157
 - 0s - loss: 1.0083 - mean_q: 36.3979
Epoch 158/158
 - 0s - loss: 1.1246 - mean_q: 36.4084
Epoch 159/159
 - 0s - loss: 1.2585 - mean_q: 36.1552
Epoch 1

Epoch 296/296
 - 0s - loss: 0.7735 - mean_q: 35.1831
Epoch 297/297
 - 0s - loss: 0.7792 - mean_q: 34.2103
Epoch 298/298
 - 0s - loss: 0.9739 - mean_q: 34.9018
Epoch 299/299
 - 0s - loss: 2.7878 - mean_q: 33.7923
Epoch 300/300
 - 0s - loss: 2.7606 - mean_q: 35.0018
Epoch 301/301
 - 0s - loss: 1.8082 - mean_q: 33.7387
Epoch 302/302
 - 0s - loss: 0.8182 - mean_q: 33.9434
Epoch 303/303
 - 0s - loss: 0.7523 - mean_q: 33.3769
Epoch 304/304
 - 0s - loss: 0.8146 - mean_q: 34.2059
Epoch 305/305
 - 0s - loss: 0.9362 - mean_q: 32.5526
Epoch 306/306
 - 0s - loss: 0.9172 - mean_q: 33.2359
Epoch 307/307
 - 0s - loss: 1.2319 - mean_q: 32.2326
Epoch 308/308
 - 0s - loss: 0.4450 - mean_q: 33.0891
Epoch 309/309
 - 0s - loss: 0.3341 - mean_q: 32.2667
Epoch 310/310
 - 0s - loss: 1.4570 - mean_q: 32.9397
Epoch 311/311
 - 0s - loss: 0.5744 - mean_q: 32.0145
Epoch 312/312
 - 0s - loss: 3.4997 - mean_q: 32.6931
Epoch 313/313
 - 0s - loss: 0.6340 - mean_q: 32.1942
Epoch 314/314
 - 0s - loss: 1.0529 - mean_q: 3

Epoch 451/451
 - 0s - loss: 0.7651 - mean_q: 31.0162
Epoch 452/452
 - 0s - loss: 0.3125 - mean_q: 31.4892
Epoch 453/453
 - 0s - loss: 0.5051 - mean_q: 30.8783
Epoch 454/454
 - 0s - loss: 1.6800 - mean_q: 30.7295
Epoch 455/455
 - 0s - loss: 0.4950 - mean_q: 30.8335
Epoch 456/456
 - 0s - loss: 0.7100 - mean_q: 31.1685
Epoch 457/457
 - 0s - loss: 1.0339 - mean_q: 31.5481
Epoch 458/458
 - 0s - loss: 0.3581 - mean_q: 30.9959
Epoch 459/459
 - 0s - loss: 0.6165 - mean_q: 31.1794
Epoch 460/460
 - 0s - loss: 0.3033 - mean_q: 30.1805
Epoch 461/461
 - 0s - loss: 0.4115 - mean_q: 30.7683
Epoch 462/462
 - 0s - loss: 1.8536 - mean_q: 30.2661
Epoch 463/463
 - 0s - loss: 1.1384 - mean_q: 30.9044
Epoch 464/464
 - 0s - loss: 1.6897 - mean_q: 29.9442
Epoch 465/465
 - 0s - loss: 0.5833 - mean_q: 30.8708
Epoch 466/466
 - 0s - loss: 0.8119 - mean_q: 30.7697
Epoch 467/467
 - 0s - loss: 0.4456 - mean_q: 30.7665
Epoch 468/468
 - 0s - loss: 0.9467 - mean_q: 30.6257
Epoch 469/469
 - 0s - loss: 0.9046 - mean_q: 3

Epoch 606/606
 - 0s - loss: 0.3092 - mean_q: 27.4290
Epoch 607/607
 - 0s - loss: 0.3747 - mean_q: 27.0156
Epoch 608/608
 - 0s - loss: 0.3898 - mean_q: 27.3017
Epoch 609/609
 - 0s - loss: 0.1166 - mean_q: 27.0764
Epoch 610/610
 - 0s - loss: 0.4416 - mean_q: 27.0450
Epoch 611/611
 - 0s - loss: 0.6368 - mean_q: 27.6518
Epoch 612/612
 - 0s - loss: 0.3471 - mean_q: 26.7546
Epoch 613/613
 - 0s - loss: 0.4423 - mean_q: 26.9386
Epoch 614/614
 - 0s - loss: 0.5806 - mean_q: 27.2181
Epoch 615/615
 - 0s - loss: 0.3641 - mean_q: 27.8558
Epoch 616/616
 - 0s - loss: 2.1143 - mean_q: 26.9805
Epoch 617/617
 - 0s - loss: 0.5095 - mean_q: 27.2753
Epoch 618/618
 - 0s - loss: 0.4179 - mean_q: 26.7785
Epoch 619/619
 - 0s - loss: 0.5873 - mean_q: 26.9710
Epoch 620/620
 - 0s - loss: 0.4527 - mean_q: 27.1463
Epoch 621/621
 - 0s - loss: 0.9329 - mean_q: 27.0358
Epoch 622/622
 - 0s - loss: 0.7036 - mean_q: 27.4854
Epoch 623/623
 - 0s - loss: 0.7291 - mean_q: 26.9766
Epoch 624/624
 - 0s - loss: 0.6267 - mean_q: 2

...

Epoch 7782/7782
 - 0s - loss: 0.6511 - mean_q: 12.7685
Epoch 7783/7783
 - 0s - loss: 0.5902 - mean_q: 12.1866
Epoch 7784/7784
 - 0s - loss: 0.2637 - mean_q: 12.5046
Epoch 7785/7785
 - 0s - loss: 0.2250 - mean_q: 12.0162
Epoch 7786/7786
 - 0s - loss: 0.2741 - mean_q: 11.7746
Epoch 7787/7787
 - 0s - loss: 0.1871 - mean_q: 11.7874
Epoch 7788/7788
 - 0s - loss: 0.5788 - mean_q: 11.3744
Epoch 7789/7789
 - 0s - loss: 0.8366 - mean_q: 11.7398
Epoch 7790/7790
 - 0s - loss: 0.6137 - mean_q: 10.7161
Epoch 7791/7791
 - 0s - loss: 0.6157 - mean_q: 11.4205
Epoch 7792/7792
 - 0s - loss: 0.6396 - mean_q: 11.5831
Epoch 7793/7793
 - 0s - loss: 0.2777 - mean_q: 11.2029
Epoch 7794/7794
 - 0s - loss: 0.4278 - mean_q: 11.8418
Epoch 7795/7795
 - 0s - loss: 0.8078 - mean_q: 11.2225
Epoch 7796/7796
 - 0s - loss: 0.2690 - mean_q: 11.5537
Epoch 7797/7797
 - 0s - loss: 0.3874 - mean_q: 10.4654
Epoch 7798/7798
 - 0s - loss: 0.3591 - mean_q: 12.2997
Epoch 7799/7799
 - 0s - loss: 0.5832 - mean_q: 10.6987
Epoch 7800

 - 0s - loss: 5.3901 - mean_q: 11.3310
Epoch 7932/7932
 - 0s - loss: 0.5168 - mean_q: 11.2756
Epoch 7933/7933
 - 0s - loss: 0.2996 - mean_q: 11.4349
Epoch 7934/7934
 - 0s - loss: 0.4058 - mean_q: 10.9507
Epoch 7935/7935
 - 0s - loss: 0.4737 - mean_q: 10.9619
Epoch 7936/7936
 - 0s - loss: 0.4805 - mean_q: 10.6673
Epoch 7937/7937
 - 0s - loss: 0.6965 - mean_q: 11.0999
Epoch 7938/7938
 - 0s - loss: 0.5938 - mean_q: 11.3070
Epoch 7939/7939
 - 0s - loss: 0.2510 - mean_q: 10.5041
Epoch 7940/7940
 - 0s - loss: 0.3876 - mean_q: 11.7099
Epoch 7941/7941
 - 0s - loss: 0.4141 - mean_q: 9.8775
Epoch 7942/7942
 - 0s - loss: 0.5697 - mean_q: 9.6144
Epoch 7943/7943
 - 0s - loss: 0.5426 - mean_q: 10.2212
Epoch 7944/7944
 - 0s - loss: 0.6721 - mean_q: 10.9468
Epoch 7945/7945
 - 0s - loss: 0.4212 - mean_q: 11.4531
Epoch 7946/7946
 - 0s - loss: 0.5742 - mean_q: 11.4114
Epoch 7947/7947
 - 0s - loss: 0.4357 - mean_q: 10.4847
Epoch 7948/7948
 - 0s - loss: 0.1516 - mean_q: 9.3578
Epoch 7949/7949
 - 0s - loss:

Epoch 8081/8081
 - 0s - loss: 0.3369 - mean_q: 10.8473
Epoch 8082/8082
 - 0s - loss: 0.4288 - mean_q: 9.6510
Epoch 8083/8083
 - 0s - loss: 0.3605 - mean_q: 9.9422
Epoch 8084/8084
 - 0s - loss: 0.4447 - mean_q: 11.3536
Epoch 8085/8085
 - 0s - loss: 0.4409 - mean_q: 10.9258
Epoch 8086/8086
 - 0s - loss: 0.7031 - mean_q: 10.2361
Epoch 8087/8087
 - 0s - loss: 0.3574 - mean_q: 10.7027
Epoch 8088/8088
 - 0s - loss: 1.0501 - mean_q: 10.4357
Epoch 8089/8089
 - 0s - loss: 0.5535 - mean_q: 10.5631
Epoch 8090/8090
 - 0s - loss: 0.3940 - mean_q: 11.6935
Epoch 8091/8091
 - 0s - loss: 0.3522 - mean_q: 9.7481
Epoch 8092/8092
 - 0s - loss: 0.2958 - mean_q: 10.8192
Epoch 8093/8093
 - 0s - loss: 0.4665 - mean_q: 11.4350
Epoch 8094/8094
 - 0s - loss: 0.2330 - mean_q: 10.4784
Epoch 8095/8095
 - 0s - loss: 0.5158 - mean_q: 10.4485
Epoch 8096/8096
 - 0s - loss: 0.5754 - mean_q: 11.2366
Epoch 8097/8097
 - 0s - loss: 0.9635 - mean_q: 11.0940
Epoch 8098/8098
 - 0s - loss: 0.4256 - mean_q: 11.5784
Epoch 8099/80

 - 0s - loss: 0.3618 - mean_q: 11.5518
Epoch 8231/8231
 - 0s - loss: 0.3486 - mean_q: 11.3805
Epoch 8232/8232
 - 0s - loss: 0.6160 - mean_q: 11.1153
Epoch 8233/8233
 - 0s - loss: 0.4507 - mean_q: 10.5547
Epoch 8234/8234
 - 0s - loss: 0.7878 - mean_q: 11.1963
Epoch 8235/8235
 - 0s - loss: 0.3016 - mean_q: 11.4825
Epoch 8236/8236
 - 0s - loss: 0.2836 - mean_q: 11.2980
Epoch 8237/8237
 - 0s - loss: 0.2062 - mean_q: 10.3923
Epoch 8238/8238
 - 0s - loss: 0.4294 - mean_q: 10.0431
Epoch 8239/8239
 - 0s - loss: 0.7459 - mean_q: 10.1770
Epoch 8240/8240
 - 0s - loss: 0.3673 - mean_q: 10.9674
Epoch 8241/8241
 - 0s - loss: 0.4923 - mean_q: 10.3922
Epoch 8242/8242
 - 0s - loss: 0.7953 - mean_q: 10.8389
Epoch 8243/8243
 - 0s - loss: 0.0906 - mean_q: 10.8204
Epoch 8244/8244
 - 0s - loss: 0.5378 - mean_q: 10.7689
Epoch 8245/8245
 - 0s - loss: 0.6800 - mean_q: 10.9313
Epoch 8246/8246
 - 0s - loss: 0.7432 - mean_q: 10.5339
Epoch 8247/8247
 - 0s - loss: 0.3823 - mean_q: 10.8551
Epoch 8248/8248
 - 0s - lo

KeyboardInterrupt: 

Save last weights:

In [ ]:
online_network.save_weights(os.path.join(weights_folder, 'weights_last.h5f'))

In [ ]:
# Dump all scores to txt-file
with open(os.path.join(name, 'episode_scores.txt'), 'w') as file:
    for item in episode_scores:
        file.write("{}\n".format(item))

print(episode_scores)

Don't forget to check TensorBoard for fancy statistics on loss and metrics using in terminal

`tensorboard --logdir=tensorboard`

after navigating to the folder containing the created folder `tensorboard`: 

In [ ]:
weights_folder

Then visit http://localhost:6006/

## Testing model

Finally, create a function to evalutate the trained network. 
Note that we still using $\varepsilon$-greedy strategy here to prevent an agent from getting stuck. 
`test_dqn` returns a list with scores for the specified number of games.

In [ ]:
def test_dqn(env, n_games, model, nb_actions, skip_start, eps=0.05, render=False, sleep_time=0.01):
    scores = []
    for i in range(n_games):
        obs = env.reset()
        score = 0
        done = False
        for skip in range(skip_start):  # skip the start of each game (it's just freezing time before game starts)
            obs, reward, done, info = env.step(0)
            score += reward
        while not done:
            state = obs
            q_values = model.predict(np.array([state]))[0]
            action = epsilon_greedy(q_values, eps, nb_actions)
            obs, reward, done, info = env.step(action)
            score += reward
            if render:
                env.render()
                time.sleep(sleep_time)
                if done:
                    time.sleep(1)
        scores.append(score)
    return scores

In [ ]:
ngames = 5
scores = test_dqn(env, ngames, online_network, nb_actions, skip_start, render=True)
print(scores)

In [ ]:
env.close()

Results are pretty poor since the training was too short. 

Try to train DQN on a cluster. You might want to adjust some hyperparameters (increase `n_steps`, `warmup`, `copy_steps` and `eps_decay_steps`; gradually decrease learning rate during training, select appropriate `batch_size` to fit gpu memory, adjust `gamma`, switch on double dqn apporach and so on). 

You can even try to make the network deeper and/or use more than one observation as an input of neural network. For instance, using few consecutive game observations would definetely improve the results since they contain some helpful information such as monsters directions, etc. Turning off TensorBoard callback on a cluster would be a good idea too.

In [17]:
online_network.save(os.path.join(weights_folder, 'ram_model_4kk.h5f'))